In [ ]:
from mira.sources.system_dynamics import *
from mira.modeling.amr.stockflow import *
from mira.modeling.amr.petrinet import * 
from mira.metamodel import *
from mira.sources.amr.stockflow import (
    template_model_from_amr_json as stockflow_amr_to_tm,
)
from mira.sources.amr.petrinet import * 
import json
import pandas as pd

pd.set_option("display.max_colwidth", None)
scenario_4_url = "https://raw.githubusercontent.com/DARPA-ASKEM/program-milestones/main/18-month-milestone/evaluation/Epi%20Use%20Case/Scenario%204%20Supplementary/stock_flow_evaluation.mdl"

## Convert the Vensim model into a MIRA template model

In [ ]:
old_tm = template_model_from_mdl_url(scenario_4_url)

## Visualize the unmodified template model

In [ ]:
old_tm.draw_jupyter()

## Inspect the parameters, initial values, and rate laws of the template model

In [ ]:
initial_values = [initial.expression for initial in old_tm.initials.values()]
df_initials = pd.DataFrame(
    {"Stock": list(old_tm.initials.keys()), "Initial Value": initial_values}
)
df_initials

In [ ]:
param_values = [param.value for param in old_tm.parameters.values()]
df_param = pd.DataFrame(
    {
        "Parameter Name": list(old_tm.parameters.keys()),
        "Parameter Value": param_values,
    }
)
df_param

In [ ]:
rate_laws = [template.rate_law for template in old_tm.templates]
template_names = [template.name for template in old_tm.templates]
df_templates = pd.DataFrame(
    {"Template ID": template_names, "Rate Law": rate_laws}
)
df_templates

## Export the template model to stock flow amr and verify if it has been exported correctly

In [ ]:
amr = template_model_to_stockflow_json(old_tm)

In [ ]:
amr["model"]["stocks"]

In [ ]:
amr["semantics"]["ode"]["initials"]

In [ ]:
amr["semantics"]["ode"]["parameters"]

In [ ]:
amr["model"]["flows"]

In [ ]:
amr["model"]["links"]

## Import the stock flow amr back to a MIRA template model

In [ ]:
new_tm = stockflow_amr_to_tm(amr)

## Stratify the infected concept and visualize the stratified template model

In [ ]:
stratified_tm = stratify(new_tm, key="infection_status", strata=["asymptomatic","symptomatic"],concepts_to_stratify=["infected"], structure=[])
stratified_tm.draw_jupyter()

## Export the model into different frameworks

In [ ]:
frameworks = {'stockflow': template_model_to_stockflow_json, 'petrinet': template_model_to_petrinet_json}

for fkey, fun in frameworks.items():
    with open(f'scenario4_{fkey}.json', 'w') as fh:
        json.dump(fun(new_tm), fh, indent=1)